In [1]:
import sklearn
import numpy as np
import pandas as pd
import sqlite3
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
import re
import pickle
from sklearn.impute import SimpleImputer

import warnings
warnings.filterwarnings('ignore')

In [2]:
conn = sqlite3.connect('../db.sqlite3')

In [3]:
df_raw = pd.read_sql_query("SELECT * FROM scraper_auto WHERE bron='marktplaats'", conn)

In [4]:
target_names = ["bouwjaar", "kilometer_stand", "vermogen", "prijs", "is_handgeschakeld", "is_benzine", "upload_datum", "apk"]
df = df_raw[target_names]
df.describe()

,bouwjaar,kilometer_stand,vermogen,prijs,is_handgeschakeld,is_benzine
count,504.000000,503.000000,504.000000,493.000000,502.000000,504.000000
mean,2007.817460,179530.477137,80.890873,4631.584178,0.960159,0.738095
std,1.764013,58138.665225,14.013271,44930.583117,0.195780,0.440108
min,2005.000000,15570.000000,51.000000,1.000000,0.000000,0.000000
25%,2006.000000,138321.500000,75.000000,1950.000000,1.000000,0.000000
50%,2007.000000,177876.000000,78.000000,2550.000000,1.000000,1.000000
75%,2009.000000,212516.000000,84.000000,3250.000000,1.000000,1.000000
max,2014.000000,368884.000000,155.000000,999999.000000,1.000000,1.000000


In [5]:
MAX_PRICE = 30000
MIN_PRICE = 1000
df = df[(df['prijs'] > MIN_PRICE) & (df['prijs'] < MAX_PRICE)]

MIN_BOUWJAAR = 2000
df = df[df['bouwjaar'] > MIN_BOUWJAAR]

MAX_VERMOGEN = 500
df = df[df['vermogen'] < MAX_VERMOGEN]

df.describe()

,bouwjaar,kilometer_stand,vermogen,prijs,is_handgeschakeld,is_benzine
count,467.000000,466.000000,467.000000,467.000000,465.00000,467.000000
mean,2007.899358,175375.967811,80.468951,2710.588865,0.95914,0.749465
std,1.779877,54654.414129,13.350692,877.791937,0.19818,0.433786
min,2005.000000,15570.000000,51.000000,1150.000000,0.00000,0.000000
25%,2006.000000,137678.000000,75.000000,1999.000000,1.00000,0.500000
50%,2007.000000,176712.500000,78.000000,2650.000000,1.00000,1.000000
75%,2009.000000,208091.250000,84.000000,3250.000000,1.00000,1.000000
max,2014.000000,368884.000000,155.000000,6950.000000,1.00000,1.000000


In [6]:
df['upload_datum'] = pd.to_datetime(df['upload_datum'])
df['apk'] = pd.to_datetime(df['apk'])

average_upload_datum = df['upload_datum'].mean().toordinal()
average_apk =df['apk'].mean().toordinal()

def upload_datum_to_ordinal(date):
    if date is pd.NaT:
        return abs(datetime.now().toordinal() - average_upload_datum)
    else:
        return abs(datetime.now().toordinal() - date.toordinal())
    
def apk_to_ordinal(date):
    if date is pd.NaT:
        return abs(datetime.now().toordinal() - average_apk)
    else:  
        return abs(datetime.now().toordinal() - date.toordinal())

df['upload_datum'] = df['upload_datum'].apply(upload_datum_to_ordinal).astype(int)
df['apk'] = df['apk'].apply(apk_to_ordinal).astype(int)

# def parse_titel(titel):
#     cleaned = re.sub('[^a-zA-Z]+', ' ', titel.lower())
#     stripped = line = re.sub(r'\b\w{1,3}\b', '', cleaned)
    
#     return stripped

# df['titel'] = df['titel'].apply(parse_titel)

In [7]:
df = df.dropna()
#df.isna().sum()

In [8]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [9]:
df.to_pickle("./data.pkl")